# Formulation et mise en oeuvre d'éléments finis de poutres à interpolations d'Hermitte en flexion et linéaire en tension

***S. Drapier, Novembre 2024***

Les préalables à la mise en place de ces éléments finis - théorie des poutres, formulation faible, discrétisation, ... - se trouvent dans le support de cours **Mécanique des Structures et Approximations numériques** de la Majeure Mécanique enseigné dans le cursus ICM de Mines Saint-Etienne https://www.emse.fr/~drapier/index_fichiers/CoursPDF/Meca-Structu2A/SDrapier_Meca-struct-num_2024.pdf - Chapitres 1&2 pour la théorie des poutres, Chapitre 4 pour la base du flambage et des vibrations, et Chapitre 6 pour la formulation de l'élément fini d'Hermitte. 

L'aide sur les sources Python du module sont ici <button onclick="window.open('html/index.html', '_blank')"> **Hermitte_NL_Classes** </button>

Les formulations sont décrites et utilisées pour 3 cas de détermination <a id="menu"></a> 
--
1. <span style="font-size:24px;">[de la réponse en ***statique linéaire***](EF_Hermitte_Lineaire.ipynb)</span>
2. <span style="font-size:24px;">[des ***charges et modes propres de vibration***](EF_Hermitte_Vibrations.ipynb)</span>
3. <span style="font-size:24px;">[des ***charges et modes de flambage*** linéarisées](EF_Hermitte_Flambage.ipynb)</span>

Cette partie préliminaire décrit l'équilibre d'une poutre dans le plan décrite par une **cinématique de Bernoulli**, sous la **forme intégrale faible** dans un cadre linéarisé. Ensuite les **approximations cinématiques** en tension et en flexion sont présentées, et les déformations discrétisées sont exprimées. Enfin, on introduit le **passage des variables locales**, dans le repère de la poutre, aux **variables globales**, dans le repère de structure.

   
|                           |    
|--------------------------------|
| <img src="hermitte.png" alt="Figure 1.1 : EF d'Hermitte dans le plan" style="width:600px; display:inline-block; vertical-align:middle;">  |
| **Figure 1.1 : EF plan d'Hermitte en flexion & linéaire en tension** |  <a name="fig1"></a>

$$
\def\vec#1{\overrightarrow{#1}} 
\def\dint{\displaystyle \int}%
\def\dsum{\displaystyle \sum}%
\def\dsqrt{\displaystyle \sqrt}%
\def\df#1#2{{\dfrac{d #1}{d #2}}}
\def\tens#1{\underline{\underline{#1}}}
\def\vect#1{\left\{\mathbf{#1}\right\}}
\def\mat#1{\left[\mathbf{#1}\right]}
\def\vecd#1#2{\begin{Bmatrix} #1 \\ #2 \end{bmatrix}}
$$

### Cinématique de Bernoulli (sans cisaillement)
Dans le repère local de la poutre oriéntée selon $\vec x$ ($O,\vec x, \vec y$), le déplacement $\vec u(x,y)$ de tout point $M$ de la poutre peut s'écrire en fonction du torseur cinématique mesuré au centre de gravité $G$ de chaque section ($\vec u(x,y)= u(x) \vec x + v(x) \vec y$), il s'écrit:
<a name="eq1"></a>
$$
	\begin{array}{l}
		\begin{cases}
			u(x,y) = u(x)-y \ \df{v}{x} \\ v(x,y)=v(x)
		\end{cases} \xrightarrow{\epsilon(\vec u)=\frac{1}{2}\left(\tens \nabla \vec u + \tens \nabla^T \vec u \right)}
	\left\{\begin{array}{rcl}
	\epsilon_{xx}(x,y)\vec x \otimes \vec x &= & u'(x)-y \ v''(x) \\
	&=& e_x(u') - y \ \kappa_z(v'')
	\end{array} \right. \tag{1.1}
	\end{array}
$$
Le tenseur de déformations se limite ici à une seule déformation non-nulle, et elle s'écrit sous la forme d'une déformation de membrane $e_x(x)$ constante dans la section de la poutre et d'une déformation liée à la flexion et proportionelle à la courbure $\kappa_z(x)$, dépendant de l'altitude du point considéré.


A cette cinématique qu'on traduit en degrès de liberté (*ddl*s), ici les déplacements du centre de gravité ($ \vec u_G(x) = u(x) \vec x + v(x) \vec y$), on associe classiquement les contraintes généralisées exprimées également au centre de gravité
<a name="eq2"></a>
$$ \vect{\tau (x)} = \left.
	\begin{cases}
			N(x)=\dint_S{\sigma_{xx}(x)ds} \\[1em] M(x) =  \dint_S{-y\sigma_{xx}(x)ds}
	\end{cases} \right\} \tag{1.2}
$$
**Remarque** L'effort tranchant extérieur $T(x)$ existe, et il est même indispensable - *cf* PPV ci-dessous - (<a href="#eq3">Eq. 1.3</a>), mais le cisaillement ne peut pas produire d'énergie dans la cinématique de Bernoulli.

### Forme intégrale faible $\Leftrightarrow$ PPV pour champ test C.A.(0) :
On rappelle qu'en mécanique, vérifier l'équilibre d'un milieu continu dont on peut exprimer une forme intégrale de son bilan énergétique est équivalent à annuler sa formulation intégrale faible. Considérons un milieu occupant un volume $\Omega$ et soumis à des forces volumiques $\vec{f}(\vec{x})$ et des efforts répartis $\vec{F}^d(\vec{x})$ sur sa frontière de Neumann $\partial \Omega_F$, et contraint par des conditions de Dirichlet ($\vec u^d(\vec x)$) sur sa frontière $\partial \Omega_u$. Ce qui s'écrit formellement (en statique HPP) :
$$
\left|\begin{array}{l}
\text{Trouver} \vec{u}\in\;U\;\text{tel que pour tout }\delta \vec{u}\in\;V: \\[+1em]
\begin{array}{rcl} \delta \Pi(\vec{u})&=&\dint_{\Omega} {}^t\tens{\nabla}\vec{u}(\vec{x}):\tens{\tens{\mathcal{C}}}(\vec{x}):
\tens{\nabla}\delta \vec{u}(\vec{x})\;d\Omega \\[+1em] &&- \dint_{\Omega}\vec{f}(\vec{x})\cdot\delta
\vec{u}(\vec{x}) \;d\Omega-\dint_{\partial \Omega_F}\vec{F}^d(\vec{x})\cdot\delta
\vec{u}(\vec{x})\;d\omega =0\end{array} \\[+2.5em]
\text{avec } U=\left\{\vec{u}\in H^1(\Omega)/\vec{u}(\vec{x})=\vec{u}^d(\vec{x})\:\:,\forall \:\:
\vec{x} \in
\partial \Omega_u\right\} \\
\text{et } V=\left\{\delta \vec{u}\in H^1(\Omega)/\delta \vec{u}(\vec{x})=\vec{0}
\:\:,\forall \:\: \vec{x} \in
\partial \Omega_u\right\}
  \end{array} \right. 
$$

De façon équivalente, mais plus concise, pour notre poutre en dynamique l'équilibre est caractérisé par la forme intégrale faible à annuler qui équivaut au **Principe des Puissances Virtuelles** lorsque le champs test est égal à la variation du champs réel. Rappelons que le champ de déplacement réel (solution) est **Cinématiquement Admissible** par nature, *i.e.* il doit vérifier les conditions aux limites cinématiques (de Dirichlet). Si le problème est dynamique, ce champ solution doit également vérifier les **Conditions Initiales** aux temps extrêmes $t \in \{t_1,t_2 \}$; soit $\vec u(\vec x,t) \in \{C.A. \cup \ C.I.\} $. Par conséquent, la variation de ce champ réel s'annule en ces points $(\vec x,t)$ particuliers : 
<a name="eq1.3"></a>
$$\delta \vec{u}(\vec x,t) \in \{C.A.(0) \cup C.I.(0)\} / \vec{u}(\vec{x,t})=\vec{0} \:\:,\forall \:\: \vec u(\vec{x},t) \in \{C.A.(0) \cup C.I.(0)\} $$
L'équilibre de notre poutre s'exprime donc sous la forme
$$\begin{equation}
	\begin{array}{ccccc}
		\delta \mathcal{P}_{int}(\delta\vec{u}(\vec{x},t)) + \delta \mathcal{P}_{ext}(\delta\vec{u}(\vec{x},t))
		=\delta \mathcal{P}_{acc}(\delta\vec{u}(\vec{x},t))\hspace{.3cm},\forall
		\delta\vec{u}(\vec{x},t) \in C.A.(0) \cup C.I.(0)\\[+1em]
	\end{array} \tag{1.3}
\end{equation}$$
avec les puissances virtuelles définies à partir de la cinématique (<a href="#eq1.1">Eq. 1.1</a>) et des contraintes généralisées (<a href="#eq1.2">Eq. 1.2</a>):
$$\begin{equation}
	\begin{array}{rcl} 
	 \nonumber	\delta \mathcal{P}_{int}(\delta\vec{u}(x,t))&=& -\dint_{0}^{l} \left\{N(x,t)\:\delta e_x (x,t) + M(x,t)\:\delta \kappa_z(x,t) \right\}dl \\[+1em] 
		\delta \mathcal{P}_{ext}(\delta\vec{u}(x,t))&=& \dint_{0}^{l} \left\{c_z(x,t)\:\delta v'(\vec{x},t)+
		p_x(x,t)\:\delta u(\vec{x},t) + p_y(x,t)\:\delta v(\vec{x},t)\right\}dl\\
		&&+\left[ N_i(t)\delta u_i(t)+T_i(t)\delta v_i(t)+M_i(t)\:\delta v'_i(t)\right]^l_0  \\[+1em] 
		\delta \mathcal{P}_{acc}(\delta\vec{u}(x,t)) &=&\dint_{0}^{l}
		\left\{\rho S \ddot{u}(x,t) \delta u(x,t)+ \rho S \ddot{v}(x,t) \delta v(x,t) + \right. \cancel{<\rho I> \ddot{v''}(x,t) \delta v(x,t) } \}dl 
	\end{array}
\end{equation}$$
Les dérivées sont notées de façon simplifiée : dérivées spatiales $\left( \frac{d\ \Box}{dx} = \Box ' \right)$ et dérivées temporelles $\left( \frac{d\ \Box}{dt} = \dot \Box \right)$. 
#### Remarques :
- les chargements répartis travaillent avec les mêmes degrès de liberté de la poutre que les chargements ponctuels, $\{ {F}_V \} = p_x(x) \vec x + p_y(x) \vec y + c_z(x) \vec z$. 
- le travail produit par l'accélération en rotation des sections est classiquement négligé.


## Approximation du déplacement
Dans cette cinématique de Bernoulli, la courbure en flexion est proportionnelle à la dérivée seconde du déplacement transverse. L'approximation de ce déplacement doit donc être **au moins $\mathcal C^2$**, *i.e.* dérivable au moins 2 fois et dont la seconde dérivée est continue. En termes d'éléments finis, cela se traduit par une **courbure continue entre 2 éléments connexes**, ce qui assure qu'**il ne pourra y avoir d'énergie artificiellement introduite dans le maillage**.  

### Tension-compression
Pour simplifier la présentation de la formulation, on utilisera directement l'élément réel, *i.e.* sans passer par un élément de référence. Dans l'élémént réel, l'approximation de la cinématique en tension est linéaire et s'écrit dans le **repère local de l'élément $(O,\vec x, \vec y)$** :
<a name="eq1.4"></a>
$$\begin{equation}
\begin{array} {rcl}
	u(x) \vec{x} &=& \dsum^e u^e(x)\vec{x} \\[+1em]
	\text{ et } & u^e(x)& \simeq u^{e,h}(x)=u_1\;N_1(x)+u_2\;N_2(x) \\[+1em]
	&&=<N_1(x)\;,\;N_2(x)>\cdot \begin{Bmatrix} u_1 \\u_2 \end{Bmatrix} = <{N}(x)>\cdot \vect{u^e_t}
\end{array} \tag{1.4}
\end{equation}$$
avec les fonctions d'interpolation linéaires (dans l'élément réel) qui sont classiquement déterminées en prenant en compte les conditions du type $u_i=N_i(x_i)$ 
$$\begin{equation} \begin{array}{cc}
		N_1(x)=-\dfrac{x-x_2}{l_e} & N_2(x)= \dfrac{x-x_1}{l_e}
	\end{array} \end{equation}$$

### Flexion
En flexion, l'interpolation doit être au moins quadratique ($\mathcal C^2$) comme indiqué ci-dessus. On peut donc envisager une interpolation polynomiale qui s'appuiera sur 3 *ddl*s en déplacement transverse et sera donc quadratique, ce qui correspond à des éléments finis avec une interpolation polynomiale de Lagrange. Pour simplifier l'utilisation de cet élément, nous choisissons une interpolation dite d'**Hermitte**, polynomiale d'ordre 3. 
Cette approximation est construite sur 4 *ddl*s de flexion, plus spécifiquemet les déplacements transverses mesurés aux 2 noeuds de l'éléments : $v_1$ et $v_2$, mais aussi les rotations mesurées en ces points : $\theta_1$ et $\theta_2$ - *cf* <a href="#fig1.1">Figure (1)</a>. 
<a name="eq1.5"></a>
$$\begin{equation}
	\begin{array}{rcl}
	v(x,t) \ \vec{y} = \dsum^e v^e(x) \ \vec{y}\\[+0.8em]
	\text{ et } & v^e(x) & \simeq v^{e,h}(x)= \mathcal{N}_1(x) v_1 +\mathcal{N}_2(x) \theta_1 +\mathcal{N}_3(x) v_2 +\mathcal{N}_4(x)\theta_2 \\[+1em]
	v^h(x)&=&<\mathcal{N}_1(x),\mathcal{N}_2(x),\mathcal{N}_3(x),\mathcal{N}_4(x)> \cdot \begin{Bmatrix} v_1 \\ \theta_1\\ v_2 \\ \theta_2 \end{Bmatrix} \\
	&=& <\mathcal{N}(x)>\cdot \vect{u^e_f}
\end{array} \tag{1.5}
\end{equation}$$
et la rotation est égale à la dérivée du déplacement transverse (rappel : pas de cisaillement dans cet élémént fini) :
$$\begin{equation}
\theta^h(x)=\dfrac{dv^{h}}{dx}(x)=\dfrac{d\mathcal{N}_1(x)}{dx} v_1 +\dfrac{d\mathcal{N}_2(x)}{dx} \theta_1 +\dfrac{d\mathcal{N}_3(x)}{dx} v_2 +\dfrac{d\mathcal{N}_4(x)}{dx}\theta_2
\end{equation}$$
Après prises en compte des conditions sur cette approximation, les fonctions d'interpolation sont des polynômes cubiques et s'écrivent (dans l'élément réel de longueur $l_e = x_2-x_1$):
$$\begin{equation}
	\begin{array}{ll}
		\mathcal{N}_1(x)=1- 3\left(\dfrac{x}{l_e}\right)^2+2\left(\dfrac{x}{l_e}\right)^3 & %\\[+.7em]
		\mathcal{N}_2(x)=l_e\left( \left(\dfrac{x}{l_e}\right)-2\left(\dfrac{x}{l_e}\right)^2+ \left(\dfrac{x}{l_e}\right)^3\right) \\[+.7em]
		\mathcal{N}_3(x)=3\left(\dfrac{x}{l_e}\right)^2-2\left(\dfrac{x}{l_e}\right)^3   &%  \\[+.7em]
		\mathcal{N}_4(x)=-l_e\left( \left(\dfrac{x}{l_e}\right)^2-\left(\dfrac{x}{l_e}\right)^3\right) \end{array}
\end{equation}$$

### Approximation cinématique complète
Finalement, l'approximation de la cinématique est la simple addition des approximations définies indépendamment en tension et en flexion :
<a name="eq1.6"></a>
$$\begin{equation}
\begin{array}{rcl}
u \ \vec{x}+v \ \vec{y} &\simeq & \begin{bmatrix}  N_1(x) & 0 &0 & N_2(x) & 0& 0 \\  0& \mathcal{N}_1(x) & \mathcal{N}_2(x) &0 & \mathcal{N}_3(x)& \mathcal{N}_4(x) \end{bmatrix}
\cdot \begin{Bmatrix} u_1 \\ v_1\\ \theta_1\\ u_2 \\  v_2 \\ \theta_2 \end{Bmatrix} \\
&=& <\mathbf N^e(x)> \cdot \vect{q^e} \\
\end{array} \tag{1.6}
\end{equation}$$
où $\vect{q^e}$ représente le vecteur des degrés de liberté exprimés dans le repère local de la poutre et qui s'écrit bien 
$$\begin{equation}
\begin{array}{rcl}
	\vect{q^e} &= & \vect{q^e_t} + \vect{q^e_f} \\[+1em]
	\text{avec }& & \vect{q^e_t} = < u_1, 0,0, u_2,0,0>^T \text{ et } \vect{q^e_f}= < 0,  v_1, \theta_1, 0, v_2, \theta_2 >^T \\[+3em]
\end{array}
\end{equation}$$

**Remarque :**
L'autre façon de stocker les *ddl*s, $\vect{u^e_t}= <u_1,u_2>^T$ et $\vect{u^e_f}=< v_1, \theta_1, v_2, \theta_2>^T$, est bien évidemment équivalente mais permet simplement d'expliciter plus rapidement certaines écritures. 

### Déformations généralisées
Comme explicité initalement, une seule déformation est non-nulle dans la cinématique de Bernoulli, et elle s'écrit sous la forme d'une déformation de membrane $e_x(x)$ et d'une déformation liée à la courbure $\kappa_z(x)$ (<a href="#eq1.1">Eq. 1.1</a>). En introduisant les discrétisations des déplacements (<a href="#eq1.9">Eq. 1.9</a>), on peut écrire les déformations généralisées discrétisées :

$$\begin{equation}
\begin{array}{rcccc}
\vect{\epsilon^e}&=&\underbrace{\nabla < N^e(\vect{x^h})>}\vect{u^e_t} &-&y^h \ \underbrace{\nabla^2 <\mathcal N^e(\vect{x^h})>}\vect{u^e_f} \\[1em]
&=& \mat{B_L^e}_t \vect{u^e_t} &-&y \ \mat{B_L^e}_f \vect{u^e_f}
\end{array} 
\end{equation}$$

La présence du terme $"\ y \ "$ dans cette expression est liée à l'étude d'une structure, différente de l'étude d'un milieu de Cauchy car la distance du point considéré par rapport au centre de gravité de la section considérée intervient dans la decription de la flexion. Supposons pour le moment que les 2 grandeurs définissant les déformations de la section de la poutre suffisent à décrire le champ de déformation complet, on aboutit à
<a name="eq1.7"></a>
$$\begin{equation}
	\begin{array}{rcl}
		 \begin{Bmatrix}{e_x^h(x)}\\{\kappa^h_z(x)}\end{Bmatrix}&=& \begin{bmatrix}  N'_1(x) & 0 &0 & N'_2(x) & 0& 0 \\  0& \mathcal{N}''_1(x) & \mathcal{N}''_2(x) &0 & \mathcal{N}''_3(x)& \mathcal{N}''_4(x) \end{bmatrix}
		\cdot \begin{Bmatrix} u_1 \\ v_1 \\ \theta_1 \\ u_2\\  v_2 \\ \theta_2 \end{Bmatrix} \\[+2em]
		&=& \begin{bmatrix}
			\mat{\nabla <N>^e} & \mat 0 \\
			\mat 0 & \mat{\nabla^2 <\mathcal N>^e} 	\end{bmatrix}  \begin{Bmatrix}{\vect{u^e_t}}\\{\vect{u^e_f}}\end{Bmatrix} \\[+2em]
		&=&  \begin{bmatrix} \mat{B_L^e}_t & \mat 0 \\
	\mat 0 & \mat{B_L^e}_f 	\end{bmatrix}  \begin{Bmatrix}{\vect{u^e_t}}\\{\vect{u^e_f}}\end{Bmatrix} \\[+2em]
		 \text{ou }\vect{\epsilon^e} & =& \mat{B_L^e} \cdot \vect{u^e} \\[+1em]
		 		\text{dim }(2 \times 1) & = & (2 \times 6) \times (6 \times 1)
	\end{array} \tag{1.7}
\end{equation}$$

## Expressions locales et globales
La formulation intégrale faible ci-dessus (<a href="#eq1.3">Eq. 1.3</a>) s'écrit dans le repère locale de la poutre. Pourtant, cette poutre peut avoir n'importe quelle orientation par rapport au repère global, ou **repère de structure**. Il faut donc exprimer toutes les grandeurs qui interviennent dans cet équilibre dans ce repère de structure. 

Notre problème étant formulé en déplacements, le changement de base du vecteur des inconnues nodales va nous permettre d'expliciter les grandeurs. Le vecteur des inconnues nodales complet va donc changer selon la base d'expression - se référer à la <a href="#fig1.1">Figure (1.1)</a> ci-dessus pour les notations :
$$\begin{equation}
 \vect{q^e}_{(l)} = 	\begin{Bmatrix} u_1 \\ v_1\\ \theta_1\\ u_2 \\ v_2 \\ \theta_2 \end{Bmatrix}_{(\vec{x},\vec{y})}\leftrightarrow \;\;\; \begin{Bmatrix} u^1_1\\ u_2^1 \\ \theta^1\\ u^2_1 \\ u^2_2 \\ \theta^2 \end{Bmatrix}_{(\vec{e}_1,\vec{e}_2)} = \vect{q^e}_{(g)}
\end{equation}$$ 

En notant $C \equiv  \cos{\theta}$ et $S \equiv \sin{\theta}$ avec l'angle $\theta$ formé entre l'élément et le repère de structure ($O,\vec e_1, \vec e_2$) - *cf* <a href="#fig1.1">Figure (1)</a>
<a name="eq1.8"></a>
$$\begin{equation}
	\begin{array}{rcl}
		 \vect{u^e}_{(g)}= \begin{Bmatrix} u_1\\ u_2 \\ \theta \end{Bmatrix}_{(\vec{e}_1,\vec{e}_2)} &= & \begin{bmatrix} C & -S &0 \\ S & C&0 \\ 0& 0&1 \end{bmatrix} \begin{Bmatrix} u \\ v \\ \theta \end{Bmatrix}_{(\vec{x},\vec{y})}  = \mat{P} \vect{u^e}_{(l)}
\end{array} \tag{1.8}
\end{equation}$$


$$ \begin{array}{rcll}
	\vect{q^e}_{(l)}=\mat{R}^T \vect{q^e}_{(g)} &\text{ ou }& \vect{q^e}_{(l)} = \begin{Bmatrix} \begin{pmatrix} u_1\\ u_2 \\ \theta \end{pmatrix}_{(1)} \\ \begin{pmatrix} u_1\\ u_2 \\ \theta \end{pmatrix}_{(2)} \end{Bmatrix}_{(\vec{x},\vec{y})}  =& \begin{bmatrix} \mat{P}^T & \mat{0} \\ \mat{0}& \mat{P}^T \end{bmatrix}	\begin{Bmatrix}  u^1_1\\ u^1_2 \\ \theta^1 \\ u^2_1\\ u^2_2 \\ \theta^2 \end{Bmatrix}_{(\vec{e}_1,\vec{e}_2)} 	% \begin{Bmatrix} u_1\\ u_2 \\ \theta \end{Bmatrix}_{(\vec{e}_1,\vec{e}_2)} &= & \begin{bmatrix} C & -S &0 \\ S & C&0 \\ 0& 0&1 \end{bmatrix} \begin{Bmatrix} u \\ v \\ \theta \end{Bmatrix}_{(\vec{x},\vec{y})}  = \mat{R} \vect{u^e}_{(l)}
  \\
  &&\text{dim }(6 \times 1)  = & \hspace{1.5em} \overbrace{(6 \times 6)  \hspace{1.5em} \times \hspace{1.5em} (6 \times1) }
\end{array}
$$

